In [133]:
import numpy as np
import uproot
from copy import deepcopy
import os
import glob
import plotly.graph_objs as go
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact, Output
from IPython.display import display, clear_output
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.optimize import fmin
import matplotlib.pyplot as plt
import pandas as pd
import particle


In [124]:
df = uproot.open("../build/test.root:Hits")["pixelHits"].arrays(library="pd")
geom = uproot.open("../build/test.root:geometry").arrays(library="np")

In [125]:
print(df.iloc[0])
row = df.iloc[0]
# long_df = pd.DataFrame({
#     "x": row["hit_rowID"],
#     "y": row["hit_colID"],
#     "z": row["hit_layerID"],
#     "pdg": row["hit_pdgc"]
# })

long_df = pd.DataFrame({
    "x": [geom['pixel_xpos'][0][int(i)] for i in row["hit_rowID"]],
    "y": [geom['pixel_Ypos'][0][int(i)] for i in row["hit_colID"]],
    "z": [geom['pixel_Zpos'][0][int(i)] for i in row["hit_layerID"]],
    "pdg": row["hit_pdgc"],
    "isPrimary": row["hit_fromPrimaryLepton"]
})

print("Particle species: ", set(long_df["pdg"]))

event_id                                                                 0
hit_rowID                [5957.0, 2585.0, 5955.0, 8382.0, 8383.0, 8384....
hit_colID                [4281.0, 7624.0, 4280.0, 3164.0, 3164.0, 3164....
hit_layerID              [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...
hit_pdgc                 [-211, 1000140280, -211, 11, 11, 11, -11, -11,...
hit_trackID              [2, 38710, 2, 3089, 3089, 3089, 4834, 4834, 7,...
hit_parentID             [1, 38703, 1, 7, 7, 7, 3082, 3082, 6, 6, 6, 30...
hit_px                   [-199.59102, -0.0, -201.95308, -4.654566, -4.6...
hit_py                   [-128.85909, -0.0, -119.657394, -0.44359952, -...
hit_pz                   [50588.945, 0.0, 50578.008, 5.481224, 5.475238...
hit_energy               [50589.695, 26053.193, 50578.742, 7.222652, 7....
hit_charge               [-1.0, 14.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0,...
hit_fromPrimaryPizero    [False, False, False, False, False, False, Fal...
hit_fromFSLPizero        

In [126]:
def pdg_to_particle(pdg):
    if abs(pdg) == 11:
        return "electron"
    if abs(pdg) == 13:
        return "muon"
    if abs(pdg) == 15:
        return "tau"
    return "other"

long_df["particle"] = long_df["pdg"].apply(pdg_to_particle)


In [135]:
def pdg_to_name(pdg):
    return particle.Particle.from_pdgid(pdg).name

long_df["name"] = long_df["pdg"].apply(pdg_to_name)


In [127]:
def isPrim_to_particle(isPrim):
    if isPrim:
        return "Primary"
    else:
        return "Secondary"

long_df["primary"] = long_df["isPrimary"].apply(isPrim_to_particle)

In [136]:
import plotly.express as px

fig = px.scatter_3d(
    long_df,
    x="x",
    y="y",
    z="z",
    color="particle",
    color_discrete_map={
        "electron": "blue",
        "muon": "orange",
        "tau": "purple",
        "other": "gray"
    },
    opacity=0.8,
    hover_name="name"
)

fig.update_traces(marker=dict(size=1))
fig.show()


In [129]:
import plotly.express as px

fig = px.scatter_3d(
    long_df,
    x="x",
    y="y",
    z="z",
    color="primary",
    color_discrete_map={
        "Primary": "orange",
        "Secondary": "gray"
    },
    opacity=0.8,
)

fig.update_traces(marker=dict(size=1))
fig.show()
